# GEOSPACIAL VISUALIZATION PROJECT

- Mi empresa:
    - Compañia de software con 50 empleados.
    - Producto: videojuegos
    - Empleados: 20 developers, 20 Designers/Creatieves/UX/UI and 10 executives/managers
     
- Requisitos buscados para localizar la empresa: radio de cercania: 50 km

        --> offices.latitude
        --> offices.longitude
        
    - Debe haber software engineers trabajando cerca:
        - category_code: 'software','web','biotech','mobile','hardware','games_video','ecommerce','enterprise','public_relations','consulting'
    
    - Debe haber un buen ratio entre grandes compañias / startups
        - number_of_employees: 0-100 small; 100-999 medium; >1000 big
        - total_money_raised
        
    - Empresas cercanas deben cubrir los intereses de mis trabajadores --> Text mining
        - description
        - tag_list
        - twitter_username: nº de seguidores, nºtweets, text mining (topics)
        
    - Prefiero estar cerca de compañias nuevas que viejas
        - deadpooled_year: null -- ELimino las compañias que quebraron
        
        - founded_year -- considero solo compañias con 10 años de antiguedad maximo
      
    
    

CAMPOS DEL DATASET:

offices.latitude; offices.longitude; 
name, category_code
number_of_employees; total_money_raised
description; tag_list; twitter_username
founded_year; deadpooled_year: null

In [1]:
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

1. Crear un cursor con los campos que me interesan y filtrando de entrada ciertas empresas. 

In [2]:
companies_interest = db.companies.find({"deadpooled_year": None},
                                       {"_id":0,"name":1,"category_code":1,"number_of_employees":1,"ipo.valuation_amount":1,
                                       "total_money_raised":1,"description":1,
                                       "tag_list":1,"founded_year":1,"twitter_username":1,
                                       "offices.latitude":1,"offices.longitude":1})

companies_interest = list(companies_interest)

In [3]:
import pandas as pd

df_companies = pd.DataFrame(companies_interest)
df_companies.head(20)

df_companies.shape

(17872, 10)

In [4]:
df_companies.describe()

,founded_year,number_of_employees
count,12425.000000,8403.000000
mean,2002.243058,840.450315
std,13.328602,11384.038864
min,1800.000000,0.000000
25%,2001.000000,4.000000
50%,2006.000000,10.000000
75%,2008.000000,35.000000
max,2013.000000,405000.000000


In [5]:
#Change somethings in columns

df_= pd.DataFrame(pd.DataFrame(df_companies["offices"].values.flatten([0]).tolist()))
df_companies = pd.concat([df_,df_companies],axis=1)


#al aplicar flatten sobre la columna 'offices' se generan 62 columnas, la mayoria vacias. me quedo unicamente
#con la primera, que representa la primera oficina y donde la gran mayoria de las entradas presentan datos


/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
df_companies = df_companies[['name','category_code',0,'founded_year','number_of_employees',
                             'total_money_raised','description','tag_list','twitter_username']].copy()

In [7]:
df_companies.columns  = ['name','category','coords','founded_year','employees','total_money_raised',
                         'description','tags','twitter']

df_companies.head()

,name,category,coords,founded_year,employees,total_money_raised,description,tags,twitter
0,Digg,news,"{'latitude': 37.764726, 'longitude': -122.394523}",2004.0,60.0,$45M,user driven social content website,"community, social, news, bookmark, digg, techn...",digg
1,Flektor,games_video,"{'latitude': 34.025958, 'longitude': -118.379768}",NaN,NaN,$0,None,"flektor, photo, video",None
2,Postini,web,"{'latitude': 37.506885, 'longitude': -122.247573}",1999.0,NaN,$0,None,,None
3,Geni,web,"{'latitude': 34.090368, 'longitude': -118.393064}",2006.0,18.0,$16.5M,Geneology social network site,"geni, geneology, social, family, genealogy",geni
4,Fox Interactive Media,web,"{'latitude': 34.076179, 'longitude': -118.39417}",1979.0,0.0,$0,,,twitterapi


In [8]:
#elimino las filas que no tengan valor en founded year o en employees

import numpy as np

df_companies = df_companies[np.isfinite(df_companies['founded_year'])]
df_companies = df_companies[np.isfinite(df_companies['employees'])]

df_companies.isnull().sum()

name                     0
category                42
coords                1244
founded_year             0
employees                0
total_money_raised       0
description           2144
tags                   198
twitter               2651
dtype: int64

In [9]:
# cambio el tipo de dato ya que estaba como floats

df_companies['employees'] = df_companies['employees'].astype('int64')
df_companies['founded_year'] = df_companies['founded_year'].astype('int64')


In [10]:
# Creo bins con el número de empleados para diferenciar entre pequeña, mediana y gran empresa

bins = [0,10,100,999,500000]
labels = ["startup","small","medium","big"]
df_companies['tamaño_empresa'] = pd.cut(df_companies['employees'], bins=bins, labels=labels)

tamaño_count = df_companies.groupby(df_companies.tamaño_empresa)['name'].nunique()

tamaño_count

tamaño_empresa
startup    3906
small      2591
medium      674
big         211
Name: name, dtype: int64

In [11]:
# Extraer el numero en columna total money raised como indicador de tamaño de empresa

import re

def change_row (row):
    row = ''.join((ch if ch in '0123456789.' else ' ') for ch in row)
    return row

df_companies['total_money_raised'] = (df_companies['total_money_raised'].apply(change_row))

df_companies['total_money_raised(M)'] = df_companies['total_money_raised'].astype('float64')

df_companies = df_companies.reset_index()

df_companies = df_companies.drop(['total_money_raised','index'], axis=1,)

In [12]:
df_companies.head()

,name,category,coords,founded_year,employees,description,tags,twitter,tamaño_empresa,total_money_raised(M)
0,Digg,news,"{'latitude': 37.764726, 'longitude': -122.394523}",2004,60,user driven social content website,"community, social, news, bookmark, digg, techn...",digg,small,45.00
1,Geni,web,"{'latitude': 34.090368, 'longitude': -118.393064}",2006,18,Geneology social network site,"geni, geneology, social, family, genealogy",geni,small,16.50
2,Fox Interactive Media,web,"{'latitude': 34.076179, 'longitude': -118.39417}",1979,0,,,twitterapi,NaN,0.00
3,Scribd,news,"{'latitude': 37.789634, 'longitude': -122.404052}",2007,50,Read Unlimited Books,"book-subscription, digital-library, netflix-fo...",scribd,small,25.80
4,MeetMoi,social,"{'latitude': 40.757929, 'longitude': -73.985506}",2007,15,Mobile Dating,"mobile, dating, location, realtime, phone",meetmoi,small,5.58


In [13]:
df_companies.dropna(subset=['coords'], inplace=True)
df_companies.dropna(subset=['tamaño_empresa'], inplace=True)
df_companies.dropna(subset=['category'], inplace=True)


def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array['longitude'],array['latitude']]
    }


df_companies['coords'] = df_companies.apply(lambda r: toGeoJSON(r['coords']), axis=1)


df_companies.head()

,name,category,coords,founded_year,employees,description,tags,twitter,tamaño_empresa,total_money_raised(M)
0,Digg,news,"{'type': 'Point', 'coordinates': [-122.394523,...",2004,60,user driven social content website,"community, social, news, bookmark, digg, techn...",digg,small,45.00
1,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",2006,18,Geneology social network site,"geni, geneology, social, family, genealogy",geni,small,16.50
3,Scribd,news,"{'type': 'Point', 'coordinates': [-122.404052,...",2007,50,Read Unlimited Books,"book-subscription, digital-library, netflix-fo...",scribd,small,25.80
4,MeetMoi,social,"{'type': 'Point', 'coordinates': [-73.985506, ...",2007,15,Mobile Dating,"mobile, dating, location, realtime, phone",meetmoi,small,5.58
6,Plaxo,web,"{'type': 'Point', 'coordinates': [-122.055197,...",2002,50,Contact Management,"contact-management, address-book, sync, contac...",plaxo,small,28.30


In [15]:
# filtro categorias y año de fundacion (empresas que tengan maxixmo 10 años)

df_companies = df_companies.loc[df_companies['category'].isin(['news', 'web','games_video','mobile',
                                                               'photo_video', 'security', 'network_hosting',
                                                               'messaging', 'hardware', 'enterprise','software',
                                                               'ecommerce', 'public_relations', 'finance', 'music',
                                                               'analytics', 'biotech', 'sports','automotive',
                                                               'education','consulting', 'design', 'legal', 
                                                               'manufacturing'])]

df_companies = df_companies.loc[df_companies["founded_year"]>=2008]

df_companies = df_companies.reset_index()
df_companies = df_companies.drop(['index'], axis=1)
df_companies.head()

,name,category,coords,founded_year,employees,description,tags,twitter,tamaño_empresa,total_money_raised(M)
0,GoingOn,software,"{'type': 'Point', 'coordinates': [-122.392142,...",2008,40,Private Academic Networks,"socialnetworking, enterprise, social-media, so...",GoingOn_Network,small,8.5
1,Cloudant,enterprise,"{'type': 'Point', 'coordinates': [-71.05781, 4...",2008,70,NoSQL database managed service,"cloud-computing, big-data, databases, software...",cloudant,small,18.3
2,KIT digital,enterprise,"{'type': 'Point', 'coordinates': [-73.971655, ...",2008,200,video distribution solutions,,kitdigital,medium,20.0
3,YouBeQB,games_video,"{'type': 'Point', 'coordinates': [-122.151158,...",2008,8,,"youbeqb, fantasyfootball, fantasyleague, sports",,startup,0.0
4,Crunchyroll,games_video,"{'type': 'Point', 'coordinates': [-122.393229,...",2008,50,Digital Entertainment and Media Platform,"crunchyroll, anime, manga, video, gaming, musi...",crunchyroll,small,4.8


In [16]:
#elimino los registro que no tengan datos en coordenadas

df_companies = df_companies.loc[df_companies['coords']!= {'type': 'Point', 'coordinates': [None, None]}]

In [17]:
# guardar un nuevo json con las compañias seleccionadas y las coordenadas en formato geojson

df_companies.to_json("companies_clean.json", orient="records", lines=True)

In [18]:
df_companies.shape

(1043, 10)

In [29]:
df_companies['coords'][333]

{'coordinates': [-88.294854, 41.931912], 'type': 'Point'}

## Una vez el el json limpio y con las coordenadas en formato GeoJson, lo guardo y lo importo desde mongodb.

In [169]:
# Creo una lista con pares de coordenadas
list_names = list(df_companies['name'])
list_coord = list(df_companies['coords'])
list_coord = list (map(lambda d: d['coordinates'], list_coord))
list_names

dict_coord = dict(zip(list_names,list_coord))

len(dict_coord)

975

In [167]:

def near_companies(coords): # esta función devuelve un dataframe con las empresas en un radio de 50km
    cursor = db.clean_companies.find({
      "coords": {
        "$near": {
          "$geometry": {
            "type": "Point",
            "coordinates": [coords[0], coords[1]]
          },
          "$minDistance": 0,
          "$maxDistance":5000
        }
      }
    })
    
    df_geocomp = pd.DataFrame(list(cursor))
    
    return df_geocomp


def results (df_geocomp):
    numero = df_geocomp['_id'].count()
    category = df_geocomp['category'].value_counts()
    tamaño = df_geocomp['tamaño_empresa'].value_counts()
    return "Empresas:{}\nCategorias:\n{}\nTamaño:\n{}".format(str(numero),category,tamaño)


def results_df (df_geocomp):
    results_df = df_geocomp[['category','tamaño_empresa']].copy()
    return results_df


In [170]:
df_comparative= 

for key, value in dict_coord.items():
    print(key,results(near_companies(value)))

GoingOn Empresas:62
Categorias:
web                 15
games_video         10
enterprise           8
software             6
mobile               5
ecommerce            4
public_relations     2
hardware             2
finance              2
consulting           2
analytics            1
network_hosting      1
news                 1
legal                1
design               1
music                1
Name: category, dtype: int64
Tamaño:
startup    38
small      20
medium      4
Name: tamaño_empresa, dtype: int64
Cloudant Empresas:10
Categorias:
enterprise          3
web                 2
software            2
games_video         1
public_relations    1
consulting          1
Name: category, dtype: int64
Tamaño:
startup    7
small      3
Name: tamaño_empresa, dtype: int64
KIT digital Empresas:51
Categorias:
web                 18
public_relations     6
ecommerce            6
games_video          5
software             5
education            2
enterprise           2
mobile               2
new

InformedMD Empresas:1
Categorias:
web    1
Name: category, dtype: int64
Tamaño:
startup    1
Name: tamaño_empresa, dtype: int64
Magento Empresas:2
Categorias:
ecommerce    1
mobile       1
Name: category, dtype: int64
Tamaño:
startup    1
medium     1
Name: tamaño_empresa, dtype: int64
Intent Empresas:11
Categorias:
web            7
ecommerce      2
games_video    1
enterprise     1
Name: category, dtype: int64
Tamaño:
startup    7
small      3
medium     1
Name: tamaño_empresa, dtype: int64
BuyerRaja Empresas:1
Categorias:
web    1
Name: category, dtype: int64
Tamaño:
small    1
Name: tamaño_empresa, dtype: int64
Episodic Empresas:3
Categorias:
games_video    1
web            1
software       1
Name: category, dtype: int64
Tamaño:
startup    2
small      1
Name: tamaño_empresa, dtype: int64
Kalibera Empresas:2
Categorias:
web    2
Name: category, dtype: int64
Tamaño:
startup    1
small      1
Name: tamaño_empresa, dtype: int64
Noovo Empresas:27
Categorias:
web                12
mobile

GeoGraffiti Empresas:3
Categorias:
ecommerce    1
web          1
mobile       1
Name: category, dtype: int64
Tamaño:
startup    2
medium     1
Name: tamaño_empresa, dtype: int64
Syncplicity Empresas:6
Categorias:
games_video        2
web                2
enterprise         1
network_hosting    1
Name: category, dtype: int64
Tamaño:
startup    5
small      1
Name: tamaño_empresa, dtype: int64
New Relic Empresas:3
Categorias:
games_video    1
web            1
software       1
Name: category, dtype: int64
Tamaño:
startup    2
small      1
Name: tamaño_empresa, dtype: int64
Rekatu Empresas:1
Categorias:
games_video    1
Name: category, dtype: int64
Tamaño:
startup    1
Name: tamaño_empresa, dtype: int64
Jinni Empresas:4
Categorias:
games_video    3
analytics      1
Name: category, dtype: int64
Tamaño:
small    4
Name: tamaño_empresa, dtype: int64
Tensor Media Empresas:3
Categorias:
web                 1
public_relations    1
software            1
Name: category, dtype: int64
Tamaño:
startu

Pinch Media Empresas:53
Categorias:
web                 19
software             7
public_relations     6
ecommerce            6
games_video          4
education            2
enterprise           2
mobile               2
news                 1
network_hosting      1
finance              1
consulting           1
analytics            1
Name: category, dtype: int64
Tamaño:
startup    30
small      15
medium      5
big         3
Name: tamaño_empresa, dtype: int64
PRunderground Empresas:1
Categorias:
public_relations    1
Name: category, dtype: int64
Tamaño:
small    1
Name: tamaño_empresa, dtype: int64
Manam Infotech Empresas:1
Categorias:
software    1
Name: category, dtype: int64
Tamaño:
small    1
Name: tamaño_empresa, dtype: int64
Ucareer Empresas:9
Categorias:
web           3
software      2
messaging     1
mobile        1
enterprise    1
education     1
Name: category, dtype: int64
Tamaño:
startup    6
small      3
Name: tamaño_empresa, dtype: int64
DormTrader Empresas:1
Categorias:
w

CreActivity Focus Empresas:1
Categorias:
web    1
Name: category, dtype: int64
Tamaño:
startup    1
Name: tamaño_empresa, dtype: int64
Yaika Empresas:1
Categorias:
games_video    1
Name: category, dtype: int64
Tamaño:
small    1
Name: tamaño_empresa, dtype: int64
Pixelmatic Empresas:11
Categorias:
web                 3
games_video         2
mobile              2
software            2
public_relations    1
enterprise          1
Name: category, dtype: int64
Tamaño:
startup    9
small      2
Name: tamaño_empresa, dtype: int64
Forvo Empresas:1
Categorias:
web    1
Name: category, dtype: int64
Tamaño:
startup    1
Name: tamaño_empresa, dtype: int64
Uprizer Labs Empresas:8
Categorias:
software     3
web          2
messaging    1
mobile       1
education    1
Name: category, dtype: int64
Tamaño:
startup    6
small      2
Name: tamaño_empresa, dtype: int64
JumpForward Empresas:13
Categorias:
ecommerce           3
web                 3
mobile              2
software            2
public_relation

...

Handbags4Hunger Empresas:1
Categorias:
ecommerce    1
Name: category, dtype: int64
Tamaño:
startup    1
Name: tamaño_empresa, dtype: int64
THE SYSTEM ARCHITECT Empresas:3
Categorias:
web           1
mobile        1
consulting    1
Name: category, dtype: int64
Tamaño:
small      2
startup    1
Name: tamaño_empresa, dtype: int64
Samasource Empresas:62
Categorias:
web                 15
games_video         10
enterprise           8
software             6
mobile               5
ecommerce            4
public_relations     2
hardware             2
finance              2
consulting           2
analytics            1
network_hosting      1
news                 1
legal                1
design               1
music                1
Name: category, dtype: int64
Tamaño:
startup    38
small      20
medium      4
Name: tamaño_empresa, dtype: int64
Fresh ! Empresas:2
Categorias:
consulting    1
enterprise    1
Name: category, dtype: int64
Tamaño:
startup    1
small      1
Name: tamaño_empresa, dtype:

Worky Empresas:8
Categorias:
software            2
web                 2
public_relations    1
security            1
network_hosting     1
consulting          1
Name: category, dtype: int64
Tamaño:
startup    5
small      2
medium     1
Name: tamaño_empresa, dtype: int64
Varsity Outreach Empresas:8
Categorias:
web                4
network_hosting    2
software           1
enterprise         1
Name: category, dtype: int64
Tamaño:
startup    8
Name: tamaño_empresa, dtype: int64
Graticle Empresas:1
Categorias:
design    1
Name: category, dtype: int64
Tamaño:
startup    1
Name: tamaño_empresa, dtype: int64
ArcTouch Empresas:62
Categorias:
web                 15
games_video         10
enterprise           8
software             6
mobile               5
ecommerce            4
public_relations     2
hardware             2
finance              2
consulting           2
analytics            1
network_hosting      1
news                 1
legal                1
design               1
music       

ServiceSphere Empresas:2
Categorias:
consulting    1
ecommerce     1
Name: category, dtype: int64
Tamaño:
startup    1
small      1
Name: tamaño_empresa, dtype: int64
Hara Empresas:21
Categorias:
web            10
mobile          3
games_video     3
enterprise      2
consulting      1
software        1
finance         1
Name: category, dtype: int64
Tamaño:
startup    12
small       9
Name: tamaño_empresa, dtype: int64
IndiaOnAPage Empresas:3
Categorias:
software       2
games_video    1
Name: category, dtype: int64
Tamaño:
startup    2
small      1
Name: tamaño_empresa, dtype: int64
barcoo Empresas:11
Categorias:
web                3
mobile             2
ecommerce          2
consulting         1
hardware           1
network_hosting    1
enterprise         1
Name: category, dtype: int64
Tamaño:
small      8
startup    3
Name: tamaño_empresa, dtype: int64
Nalts Consulting Empresas:1
Categorias:
consulting    1
Name: category, dtype: int64
Tamaño:
startup    1
Name: tamaño_empresa, dtype:

BetKlub Empresas:16
Categorias:
web            8
software       3
games_video    2
news           1
mobile         1
enterprise     1
Name: category, dtype: int64
Tamaño:
startup    11
small       5
Name: tamaño_empresa, dtype: int64
B32TRUST Empresas:17
Categorias:
web            8
software       3
games_video    2
news           1
mobile         1
enterprise     1
messaging      1
Name: category, dtype: int64
Tamaño:
startup    12
small       5
Name: tamaño_empresa, dtype: int64
Enternships Empresas:22
Categorias:
web            9
games_video    3
mobile         2
software       2
enterprise     2
consulting     1
sports         1
ecommerce      1
music          1
Name: category, dtype: int64
Tamaño:
startup    17
small       5
Name: tamaño_empresa, dtype: int64
Kimengi Empresas:9
Categorias:
web            3
consulting     2
software       2
games_video    1
mobile         1
Name: category, dtype: int64
Tamaño:
startup    8
small      1
Name: tamaño_empresa, dtype: int64
oneDrum Emp

Jelli Empresas:9
Categorias:
web            3
enterprise     2
software       2
games_video    1
mobile         1
Name: category, dtype: int64
Tamaño:
startup    5
small      3
medium     1
Name: tamaño_empresa, dtype: int64
Tiny Speck Empresas:11
Categorias:
web                 3
games_video         2
mobile              2
software            2
public_relations    1
enterprise          1
Name: category, dtype: int64
Tamaño:
startup    9
small      2
Name: tamaño_empresa, dtype: int64
SEO Sumo Empresas:1
Categorias:
public_relations    1
Name: category, dtype: int64
Tamaño:
startup    1
Name: tamaño_empresa, dtype: int64
acrossair Empresas:24
Categorias:
web                 9
games_video         3
mobile              2
consulting          2
software            2
enterprise          2
public_relations    1
sports              1
ecommerce           1
music               1
Name: category, dtype: int64
Tamaño:
startup    19
small       5
Name: tamaño_empresa, dtype: int64
MyChances Empresa